In [3]:
import pandas as pd
import numpy as np
import time
import os
import warnings
import traceback
from datetime import datetime

# Feature Engineering Imports
import pandas_ta as ta  # Technical indicators (Needed for feature generation below)

# Feature Selection Imports
from sklearn.feature_selection import VarianceThreshold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.sm_exceptions import PerfectSeparationError # For handling VIF errors
from sklearn.impute import SimpleImputer # Needed for VIF imputation

# --- Suppress Warnings ---
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore') # General suppression

# --- Configuration ---
CSV_FILE_PATH = r'C:\Users\mason\AVP\BTCUSDrec.csv' # Use raw string for Windows paths

# Feature Selection Thresholds
VARIANCE_THRESHOLD = 0.0001  # Remove features with zero or very low variance
VIF_THRESHOLD = 4.19      # Remove features iteratively with VIF >= this value (Using 5 as a start)

# Define columns that are NOT features (Identifiers, Raw Data)
# Add any other non-feature columns generated by the function below
NON_FEATURE_COLS_SCRIPT_B = [
    'unix', 'date', 'symbol', 'open', 'high', 'low', 'close',
    'Volume BTC', 'Volume USD', # Keep original names as they might be used
    'timestamp', # Added if generated
    'price_change_1h', # Intermediate
    'price_change_12h', # Target from Script B
    'price_return_1h_feat', # Intermediate if not dropped inside function
    'prev_close', 'high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close', 'true_range' # ATR intermediates
]

# --- Feature Engineering Function (Copied from your "Simpler Script B" - CORRECTED TIME FEATURES) ---
def calculate_script_b_features(df):
    """
    Generates the feature set used in the 'Simpler Script B'.
    Based on the provided script structure.
    """
    print("Generating features based on 'Simpler Script B' logic...")
    df = df.copy() # Work on a copy

    # --- Ensure 'timestamp' exists EARLY ---
    if 'timestamp' not in df.columns:
        if 'date' in df.columns:
             print("  Generating 'timestamp' from 'date' column.")
             df['timestamp'] = pd.to_datetime(df['date'])
        elif 'unix' in df.columns:
             print("  Generating 'timestamp' from 'unix' column.")
             df['timestamp'] = pd.to_datetime(df['unix'], unit='ms') # Assuming unix is ms
        else:
             print("Error: Cannot find suitable column ('timestamp', 'date', or 'unix') for time features.")
             return pd.DataFrame()
    else:
        # Ensure timestamp is datetime type if it already exists
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Sort by timestamp after ensuring it exists and is datetime
    df = df.sort_values('timestamp').reset_index(drop=True) # Reset index after sort

    # --- Ensure necessary OHLCV columns exist ---
    required = ['open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD']
    if not all(col in df.columns for col in required):
        missing = [col for col in required if col not in df.columns]
        print(f"Error: Missing required base columns for feature generation: {missing}")
        return pd.DataFrame() # Return empty if base columns missing
    # Convert to numeric AFTER ensuring they exist
    for col in required:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=required) # Drop if conversion creates NaNs
    if df.empty: return pd.DataFrame()


    # --- (Rest of the feature calculations: Basic, Volatility, MAs, Lags, etc. go here as before) ---
    # ...
    # Basic Features
    df['price_change_1h'] = df['close'].pct_change() * 100
    df['price_range_pct'] = (df['high'] - df['low']) / df['close'].replace(0, np.nan) * 100
    df['oc_change_pct'] = (df['close'] - df['open']) / df['open'].replace(0, np.nan) * 100

    # Volatility Features
    def garman_klass_volatility(open_, high, low, close, window):
        log_hl = np.log(np.maximum(high, 1e-9) / np.maximum(low, 1e-9))
        log_co = np.log(np.maximum(close, 1e-9) / np.maximum(open_, 1e-9))
        gk = 0.5 * (log_hl ** 2) - (2*np.log(2) - 1) * (log_co ** 2)
        rolling_mean = gk.rolling(window=window).mean()
        rolling_mean = rolling_mean.clip(lower=0)
        return np.sqrt(rolling_mean)

    def parkinson_volatility(high, low, window):
        log_hl_sq = np.log(np.maximum(high, 1e-9) / np.maximum(low, 1e-9)) ** 2
        rolling_sum = log_hl_sq.rolling(window=window).sum()
        factor = 1 / (4 * np.log(2) * window)
        return np.sqrt(factor * rolling_sum)

    df['garman_klass_12h'] = garman_klass_volatility(df['open'], df['high'], df['low'], df['close'], window=12)
    df['parkinson_3h'] = parkinson_volatility(df['high'], df['low'], window=3)

    # Moving Averages & Standard Deviations
    df['ma_3h'] = df['close'].rolling(window=3).mean()
    df['rolling_std_3h'] = df['close'].rolling(window=3).std()

    # Lagged Features
    df['price_return_1h_feat'] = df['close'].pct_change()
    lag_periods_price = [3, 6, 12, 24, 48, 72, 168]
    for lag in lag_periods_price:
        df[f'lag_{lag}h_price_return'] = df['price_return_1h_feat'].shift(lag) * 100 # Correct shift

    df['volume_return_1h'] = df['Volume BTC'].pct_change() * 100
    lag_periods_volume = [3, 6, 12, 24]
    for lag in lag_periods_volume:
         df[f'lag_{lag}h_volume_return'] = df['volume_return_1h'].shift(lag) # Correct shift

    # Longer MAs and STDs
    ma_periods = [6, 12, 24, 48, 72, 168]
    for p in ma_periods:
        df[f'ma_{p}h'] = df['close'].rolling(window=p).mean()
    std_periods = [6, 12, 24, 48, 72, 168]
    for p in std_periods:
        df[f'rolling_std_{p}h'] = df['price_return_1h_feat'].rolling(window=p).std() * 100

    # ATR
    df['prev_close'] = df['close'].shift(1)
    df['high_minus_low'] = df['high'] - df['low']
    df['high_minus_prev_close'] = np.abs(df['high'] - df['prev_close'])
    df['low_minus_prev_close'] = np.abs(df['low'] - df['prev_close'])
    df['true_range'] = df[['high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close']].max(axis=1)
    atr_periods = [14, 24, 48]
    for p in atr_periods:
         df[f'atr_{p}h'] = df['true_range'].rolling(window=p).mean()
    # Keep intermediate ATR columns out for now
    # df = df.drop(columns=['prev_close', 'high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close', 'true_range'])

    # Trend/Interaction Features
    epsilon = 1e-9
    for p in [24, 48, 168]:
        if f'ma_{p}h' in df.columns: df[f'close_div_ma_{p}h'] = df['close'] / (df[f'ma_{p}h'] + epsilon)
    if 'ma_12h' in df.columns and 'ma_48h' in df.columns: df[f'ma12_div_ma48'] = df['ma_12h'] / (df['ma_48h'] + epsilon)
    if 'ma_24h' in df.columns and 'ma_168h' in df.columns: df[f'ma24_div_ma168'] = df['ma_24h'] / (df['ma_168h'] + epsilon)
    if 'rolling_std_12h' in df.columns and 'rolling_std_72h' in df.columns: df['std12_div_std72'] = df['rolling_std_12h'] / (df['rolling_std_72h'] + epsilon)
    if 'price_range_pct' in df.columns: df['volume_btc_x_range'] = df['Volume BTC'] * df['price_range_pct']

    # Non-linear Transformations
    if 'rolling_std_3h' in df.columns: df['rolling_std_3h_sq'] = df['rolling_std_3h'] ** 2
    if 'price_return_1h_feat' in df.columns: df['price_return_1h_sq'] = df['price_return_1h_feat'] ** 2 * 10000
    if 'rolling_std_12h' in df.columns: df['rolling_std_12h_sqrt'] = np.sqrt(df['rolling_std_12h'].clip(lower=0) + epsilon)

    # Drop intermediate price return AFTER it's used for std dev calc
    if 'price_return_1h_feat' in df.columns:
        df = df.drop(columns=['price_return_1h_feat'])

    # --- CORRECTED Time Feature Generation ---
    if 'timestamp' in df.columns: # Check if timestamp column exists
        if not any(col.startswith('hour_') for col in df.columns):
            print("  Generating hour features from 'timestamp'.")
            hour_of_day = df['timestamp'].dt.hour # Use .dt accessor on timestamp Series
            for hour in range(24): df[f'hour_{hour}'] = (hour_of_day == hour).astype(int)
        if not any(col.startswith('day_') for col in df.columns):
            print("  Generating day features from 'timestamp'.")
            day_of_week = df['timestamp'].dt.dayofweek # Use .dt accessor on timestamp Series
            for day in range(7): df[f'day_{day}'] = (day_of_week == day).astype(int)
    else:
        print("  Warning: Cannot generate time features, 'timestamp' column not found.")
    # --- END CORRECTION ---

    print("Feature generation complete.")
    return df

# --- (Keep the calculate_vif function as it was) ---
def calculate_vif(df_features, vif_threshold=10.0, verbose=True):
    # ... (VIF function code remains the same) ...
    if not isinstance(df_features, pd.DataFrame) or df_features.empty:
        print("VIF calculation skipped: Input is not a valid DataFrame or is empty.")
        return []

    if verbose: print(f"\n--- Feature Selection: Calculating VIF (Threshold: {vif_threshold}) ---")
    # Select only numeric types for VIF calculation
    features = df_features.select_dtypes(include=np.number).copy()
    if features.empty:
         print("VIF calculation skipped: No numeric features found.")
         return []

    original_feature_count = features.shape[1]
    if verbose: print(f"Numeric features before VIF check: {original_feature_count}")

    # Handle NaNs robustly
    num_nans_before = features.isnull().sum().sum()
    if num_nans_before > 0:
        if verbose: print(f"  Warning: Found {num_nans_before} NaNs. Imputing with column medians for VIF calculation ONLY.")
        try:
            imputer = SimpleImputer(strategy='median')
            features_imputed_array = imputer.fit_transform(features)
            features = pd.DataFrame(features_imputed_array, columns=features.columns, index=features.index)
        except Exception as e_imp:
             print(f"  ERROR during NaN imputation: {e_imp}. VIF calculation aborted.")
             return df_features.columns.tolist() # Return original as fallback

        num_nans_after = features.isnull().sum().sum()
        if num_nans_after > 0:
             print(f"  ERROR: {num_nans_after} NaNs remain after median imputation. This should not happen.")
             return df_features.columns.tolist() # Fallback

    # Check for infinities again after potential imputation edge cases
    num_infs = np.isinf(features.values).sum()
    if num_infs > 0:
        print(f"  ERROR: Found {num_infs} infinite values after imputation. Cannot calculate VIF. Aborting.")
        return df_features.columns.tolist()

    # Check for constant columns after imputation
    constant_cols = features.columns[features.apply(lambda x: x.nunique()) <= 1].tolist()
    if constant_cols:
        if verbose: print(f"  Warning: Removing constant columns found after imputation before VIF: {constant_cols}")
        features = features.drop(columns=constant_cols)
        if features.empty:
            print("  ERROR: DataFrame empty after dropping constant columns post-imputation. VIF aborted.")
            return []

    # Initial list of features to check
    kept_features = features.columns.tolist()
    dropped_features_vif = []

    # Iterative VIF calculation
    while len(kept_features) > 1:
        # Add constant term for VIF calculation inside the loop with current features
        X_vif = features[kept_features]
        try:
            # Calculate VIF
            vif_series = pd.Series(
                [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])],
                index=kept_features,
                dtype=float
            )
        except PerfectSeparationError:
            print("  Error: Perfect separation detected during VIF calculation.")
            print("  Stopping VIF calculation. Manual inspection recommended.")
            break
        except ValueError as ve:
             print(f"  ValueError during VIF calculation: {ve}.")
             break
        except Exception as e:
            print(f"  Unexpected error calculating VIF: {e}")
            break

        max_vif = vif_series.max()

        # Check for NaN/Inf VIF values
        is_nan_inf = vif_series.isna() | np.isinf(vif_series)
        if is_nan_inf.any():
             nan_inf_vif_features = vif_series[is_nan_inf].index.tolist()
             if verbose: print(f"  Warning: Found NaN/Inf VIF for: {nan_inf_vif_features}. Removing first one.")
             feature_to_drop_inf = nan_inf_vif_features[0]
             if verbose: print(f"  Dropping feature '{feature_to_drop_inf}' due to NaN/Inf VIF.")
             kept_features.remove(feature_to_drop_inf)
             dropped_features_vif.append(feature_to_drop_inf)
             continue # Recalculate

        # Check threshold
        if max_vif < vif_threshold:
            if verbose: print(f"  Max VIF ({max_vif:.2f}) is below threshold {vif_threshold}. Stopping.")
            break
        else:
            # Drop feature with highest VIF
            feature_to_drop = vif_series.idxmax()
            if verbose: print(f"  Dropping '{feature_to_drop}' (VIF: {max_vif:.2f})")
            kept_features.remove(feature_to_drop)
            dropped_features_vif.append(feature_to_drop)

    if verbose:
        print(f"Removed {len(dropped_features_vif)} features based on VIF.")
        if dropped_features_vif: print(f"  Features Dropped by VIF: {sorted(list(dropped_features_vif))}")
        print(f"Features remaining after VIF check: {len(kept_features)}")
        print(f"--- VIF calculation finished. ---")

    # Return the list of features from the original input that survived
    final_kept_feature_names = [col for col in df_features.columns if col in kept_features]
    return final_kept_feature_names

# --- Main Execution Block ---
if __name__ == "__main__":

    print("--- 1. Data Loading & Preprocessing ---")
    try:
        print(f"Loading data from: {CSV_FILE_PATH}")
        col_names = ['unix', 'date', 'symbol_csv', 'open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD']
        df_raw = pd.read_csv(CSV_FILE_PATH, header=0, names=col_names)
        print(f"Raw data loaded. Shape: {df_raw.shape}")

        # --- ENSURE timestamp IS CREATED CORRECTLY ---
        if 'date' in df_raw.columns:
             print("Using 'date' column for timestamp.")
             df_raw['timestamp'] = pd.to_datetime(df_raw['date'])
             cols_to_drop_init = ['unix', 'date', 'symbol_csv']
        elif 'unix' in df_raw.columns:
             print("Using 'unix' column for timestamp (assuming milliseconds).")
             df_raw['timestamp'] = pd.to_datetime(df_raw['unix'], unit='ms')
             cols_to_drop_init = ['unix', 'symbol_csv']
        else:
             print("Error: Neither 'date' nor 'unix' column found for timestamp generation.")
             exit()

        df_raw = df_raw.drop(columns=cols_to_drop_init, errors='ignore')
        df_raw = df_raw.sort_values('timestamp').reset_index(drop=True)
        essential_raw = ['timestamp', 'open', 'high', 'low', 'close', 'Volume BTC', 'Volume USD']
        essential_raw = [col for col in essential_raw if col in df_raw.columns]
        df_raw = df_raw[essential_raw]

        if df_raw.empty: exit("DataFrame empty after initial processing. Exiting.")
        print(f"Data prepared for feature engineering. Shape: {df_raw.shape}")

    except Exception as e: print(f"Error loading data: {e}"); traceback.print_exc(); exit()


    print("\n--- 2. Feature Engineering (Script B Logic) ---")
    feature_calc_start = time.time()
    df_full_features = calculate_script_b_features(df_raw) # Call your function
    feature_calc_end = time.time()

    if df_full_features.empty: exit("Feature calculation failed. Exiting.")
    print(f"Feature calculation completed in {feature_calc_end - feature_calc_start:.2f} seconds.")
    print(f"Total columns after feature engineering: {len(df_full_features.columns)}")

    # --- 3. Identify Potential Features ---
    # Define NON_FEATURE_COLS based on Script B's potential output
    NON_FEATURE_COLS_SCRIPT_B = [
        'unix', 'date', 'symbol', 'open', 'high', 'low', 'close',
        'Volume BTC', 'Volume USD', # Original names
        'volume_btc', 'volume_usd', # Internal names if kept
        'timestamp',
        'price_change_1h', # Intermediate
        'price_change_12h', # Target from Script B (might not be generated here)
        # ATR intermediates (check if dropped in your function)
        'prev_close', 'high_minus_low', 'high_minus_prev_close', 'low_minus_prev_close', 'true_range'
    ]
    all_potential_features = [col for col in df_full_features.columns if col not in NON_FEATURE_COLS_SCRIPT_B]
    print(f"Identified {len(all_potential_features)} potential features from Script B generation.")

    # Create DataFrame with only these potential features
    df_features_only = df_full_features[all_potential_features].copy()

    # --- ** NEW STEP 3.5: Handle Infinities BEFORE Filtering ** ---
    print("\n--- 3.5 Cleaning Infinities (BEFORE Filtering) ---")
    numeric_feature_cols = df_features_only.select_dtypes(include=np.number).columns
    infinite_counts = np.isinf(df_features_only[numeric_feature_cols]).sum()
    total_infinite = infinite_counts.sum()
    if total_infinite > 0:
        print(f"Found {total_infinite} infinite values. Replacing with NaN BEFORE filtering.")
        # print("Infinite counts per column:\n", infinite_counts[infinite_counts > 0]) # Optional detail
        df_features_only = df_features_only.replace([np.inf, -np.inf], np.nan)
        print("Infinite values replaced with NaN.")
    else:
        print("No infinite values found in features.")
    # --- END NEW STEP ---


    # --- 4. Feature Selection: Variance Threshold ---
    print(f"\n--- 4. Feature Selection: Variance Threshold (Threshold: {VARIANCE_THRESHOLD}) ---")
    features_before_variance = df_features_only.columns.tolist()
    print(f"Features before variance check: {len(features_before_variance)}")

    # Impute NaNs temporarily FOR FITTING ONLY
    df_temp_imputed_var = df_features_only.copy()
    num_nans_variance = df_temp_imputed_var.isnull().sum().sum()
    if num_nans_variance > 0:
        print(f"  Imputing {num_nans_variance} NaNs with medians temporarily for VarianceThreshold fitting.")
        try:
            # Impute only numeric columns
            numeric_cols_var = df_temp_imputed_var.select_dtypes(include=np.number).columns
            imputer_var = SimpleImputer(strategy='median')
            df_temp_imputed_var[numeric_cols_var] = imputer_var.fit_transform(df_temp_imputed_var[numeric_cols_var])
        except Exception as e_imp_var:
             print(f"  ERROR during NaN imputation for Variance Threshold: {e_imp_var}. Skipping VT.")
             features_after_variance = features_before_variance # Keep all if imputation fails
             df_features_selected = df_features_only.copy() # Use original features
             # Jump directly to VIF section
             # Note: This means VIF will likely fail too if imputation failed here.
             # Consider adding more robust error handling or stopping if imputation fails.

    # Check for remaining NaNs after imputation (shouldn't happen with median if no all-NaN columns)
    if df_temp_imputed_var.isnull().sum().sum() > 0:
        print("  Warning: NaNs still present after median imputation for VT fit. Dropping all-NaN columns.")
        df_temp_imputed_var.dropna(axis=1, how='all', inplace=True)

    # Check for constant columns AFTER imputation
    constant_cols_var = df_temp_imputed_var.columns[df_temp_imputed_var.nunique() <= 1].tolist()
    if constant_cols_var:
        print(f"  Removing constant columns found after imputation before VT fit: {constant_cols_var}")
        df_temp_imputed_var = df_temp_imputed_var.drop(columns=constant_cols_var)

    # Apply Variance Threshold if imputation succeeded
    if 'imputer_var' in locals(): # Check if imputation was attempted and likely succeeded
        try:
            selector = VarianceThreshold(threshold=VARIANCE_THRESHOLD)
            # Fit only on columns remaining after NaN/constant checks
            valid_cols_for_fit = df_temp_imputed_var.columns.tolist()
            selector.fit(df_temp_imputed_var[valid_cols_for_fit])
            # Apply mask back to the list of columns that were actually fit
            features_after_variance = [col for col, support in zip(valid_cols_for_fit, selector.get_support()) if support]
            # Determine dropped based on the set difference from before constant removal
            dropped_variance = set(features_before_variance) - set(features_after_variance)
            print(f"Removed {len(dropped_variance)} features with variance <= {VARIANCE_THRESHOLD}.")
            if dropped_variance: print(f"  Dropped by Variance Threshold: {sorted(list(dropped_variance))}")
        except Exception as e_var:
            print(f"Error during variance thresholding: {e_var}. Skipping VT.")
            features_after_variance = features_before_variance # Keep all if error
    else: # Imputation failed earlier
        features_after_variance = features_before_variance

    print(f"Features remaining after variance check: {len(features_after_variance)}")
    # Select columns in the original df_features_only that passed the variance check
    df_features_selected = df_features_only[features_after_variance].copy()


    # --- 5. Feature Selection: VIF Threshold ---
    # Pass the DataFrame containing only features that passed the variance check
    features_after_vif = calculate_vif(
        df_features_selected, # Pass the dataframe with features selected so far
        vif_threshold=VIF_THRESHOLD,
        verbose=True
    )

    # --- 6. Final Selected Features ---
    print(f"\n--- 6. Final Feature Selection Summary ---")
    print(f"Total potential features from Script B: {len(all_potential_features)}")
    print(f"Features after Variance Threshold ({VARIANCE_THRESHOLD}): {len(features_after_variance)}")
    print(f"Features after VIF Threshold ({VIF_THRESHOLD}):      {len(features_after_vif)}")
    print("\nFinal list of selected feature names (to use in modeling script):")
    if features_after_vif:
        print("\nSELECTED_FEATURE_NAMES = [")
        for i, feature in enumerate(sorted(features_after_vif)):
             print(f"    '{feature}'," + ("" if i == len(features_after_vif) - 1 else ""))
        print("]")
    else: print("  No features remained after filtering.")
    print(f"\nTotal selected features: {len(features_after_vif)}")
    print("\nFeature selection script finished.")

--- 1. Data Loading & Preprocessing ---
Loading data from: C:\Users\mason\AVP\BTCUSDrec.csv
Raw data loaded. Shape: (15177, 9)
Using 'date' column for timestamp.
Data prepared for feature engineering. Shape: (15177, 7)

--- 2. Feature Engineering (Script B Logic) ---
Generating features based on 'Simpler Script B' logic...
  Generating hour features from 'timestamp'.
  Generating day features from 'timestamp'.
Feature generation complete.
Feature calculation completed in 0.03 seconds.
Total columns after feature engineering: 87
Identified 74 potential features from Script B generation.

--- 3.5 Cleaning Infinities (BEFORE Filtering) ---
Found 10 infinite values. Replacing with NaN BEFORE filtering.
Infinite values replaced with NaN.

--- 4. Feature Selection: Variance Threshold (Threshold: 0.0001) ---
Features before variance check: 74
  Imputing 1697 NaNs with medians temporarily for VarianceThreshold fitting.
Removed 2 features with variance <= 0.0001.
  Dropped by Variance Threshold